In [1]:
import torch
from torch_geometric.data import Data

In [2]:
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

In [3]:
data = Data(x=x, edge_index=edge_index.t().contiguous())

In [7]:
from torch_geometric.datasets import TUDataset
dataset = TUDataset(root='/tmp/ENZYMES', name = 'ENZYMES')
print(len(dataset))
print(dataset.num_classes)
print(dataset.num_node_features)

Extracting /tmp/ENZYMES/ENZYMES/ENZYMES.zip
Processing...


600
6
3


Done!


In [9]:
data = dataset[0]
print(Data(edge_index=[2, 168], x=[37, 3], y=[1]))
data.is_undirected()

Data(x=[2], edge_index=[2], y=[1])


True

In [12]:
from torch_geometric.loader import DataLoader
loader = DataLoader(dataset, batch_size=32, shuffle=True)
for batch in loader:
    print(batch)
    print(batch.num_graphs)

DataBatch(edge_index=[2, 4026], x=[1031, 3], y=[32], batch=[1031], ptr=[33])
32
DataBatch(edge_index=[2, 3508], x=[924, 3], y=[32], batch=[924], ptr=[33])
32
DataBatch(edge_index=[2, 3944], x=[1034, 3], y=[32], batch=[1034], ptr=[33])
32
DataBatch(edge_index=[2, 3842], x=[970, 3], y=[32], batch=[970], ptr=[33])
32
DataBatch(edge_index=[2, 4064], x=[1071, 3], y=[32], batch=[1071], ptr=[33])
32
DataBatch(edge_index=[2, 3590], x=[940, 3], y=[32], batch=[940], ptr=[33])
32
DataBatch(edge_index=[2, 4430], x=[1136, 3], y=[32], batch=[1136], ptr=[33])
32
DataBatch(edge_index=[2, 4052], x=[1076, 3], y=[32], batch=[1076], ptr=[33])
32
DataBatch(edge_index=[2, 4164], x=[1136, 3], y=[32], batch=[1136], ptr=[33])
32
DataBatch(edge_index=[2, 4414], x=[1170, 3], y=[32], batch=[1170], ptr=[33])
32
DataBatch(edge_index=[2, 3878], x=[958, 3], y=[32], batch=[958], ptr=[33])
32
DataBatch(edge_index=[2, 4420], x=[1190, 3], y=[32], batch=[1190], ptr=[33])
32
DataBatch(edge_index=[2, 4216], x=[1131, 3], y=[

In [20]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root = '/tmp/Cora', name='Cora')

class GCN(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.conv1 = GCNConv(dataset.num_node_features, 16)
      self.conv2 = GCNConv(16, dataset.num_classes)
    def forward(self, data):
      x, edge_index = data.x, data.edge_index
      x = self.conv1(x, edge_index)
      x = F.relu(x)
      x = F.dropout(x, training=self.training) 
      x = self.conv2(x, edge_index)

      return F.log_softmax(x, dim=1)

In [21]:
device = torch.device('cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(400):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [22]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct)/int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8000
